In [16]:
elastic_rods_dir = '../../../elastic_rods/python/'
weaving_dir = '../../'
import os
import os.path as osp
import sys; sys.path.append(elastic_rods_dir); sys.path.append(weaving_dir)
import numpy as np, elastic_rods, linkage_vis
import numpy.linalg as la
from bending_validation import suppress_stdout as so
import matplotlib.pyplot as plt
from elastic_rods import EnergyType, InterleavingType

# weaving
import analysis_helper, ribbon_linkage_helper, mesh_vis, linkage_utils, compute_curve_from_curvature, pipeline_helper, importlib
importlib.reload(analysis_helper)
importlib.reload(ribbon_linkage_helper)
importlib.reload(mesh_vis)
importlib.reload(linkage_utils)
importlib.reload(compute_curve_from_curvature)
importlib.reload(pipeline_helper)
from analysis_helper import (compare_turning_angle,
                            is_on_sphere, 
                            get_distance_to_center_scalar_field, 
                            plot_curvatures, 
                            get_curvature_scalar_field,
                            construct_elastic_rod_loop_from_rod_segments, 
                            concatenate_rod_properties_from_rod_segments, 
                            compute_min_distance_rigid_transformation)
from ribbon_linkage_helper import (update_rest_curvature, 
                                   set_ribbon_linkage,
                                   export_linkage_geometry_to_obj,
                                   write_linkage_ribbon_output_florin)
from compute_curve_from_curvature import (match_geo_curvature_and_edge_len, get_all_curve_pattern)
from linkage_utils import order_segments_by_ribbons, get_turning_angle_and_length_from_ordered_rods

from pipeline_helper import (initialize_linkage, get_normal_deviation, set_joint_vector_field, stage_1_optimization, initialize_stage_2_optimizer, stage_2_optimization, InputOrganizer, write_all_output, set_surface_view_options, get_structure_analysis_view, get_double_side_view, show_selected_joints)
import vis.fields
import matplotlib.cm as cm
import time

In [3]:
rod_length = 0.1756505993328395
width = rod_length / 15 * 5
thickness = width / 5 * 0.35
print(width, thickness)
io = InputOrganizer('regular_torus_1', thickness, width, weaving_dir)


0.058550199777613166 0.004098513984432921


In [4]:
OPTS = elastic_rods.NewtonOptimizerOptions()
OPTS.gradTol = 1e-6
OPTS.verbose = 1;
OPTS.beta = 1e-8
OPTS.niter = 200
OPTS.verboseNonPosDef = False
rw = 0.01
sw = 0.01

In [27]:
with so(): curved_linkage = initialize_linkage(surface_path = io.SURFACE_PATH, useCenterline = True, model_path = io.MODEL_PATH, cross_section = io.RIBBON_CS, subdivision_res = io.SUBDIVISION_RESOLUTION)
curved_linkage.set_design_parameter_config(use_restLen = True, use_restKappa = True)
curved_save_tgt_joint_pos = curved_linkage.jointPositions();
curved_linkage_view = get_double_side_view(curved_linkage, flip = False)
curved_linkage_view.show()

Renderer(camera=PerspectiveCamera(aspect=1.6, children=(PointLight(color='white', intensity=0.6, position=(0.0…

In [28]:
curved_linkage.setExtendedDoFsPSRL(np.load('regular_torus_1_dof.npy'))

In [32]:
topColor =[180/255., 158/255., 119/255.]
bottomColor =[79/255., 158/255., 246/255.]
heights = curved_linkage.visualizationGeometryHeightColors()
colors = np.take(np.array([bottomColor, topColor]), heights < heights.mean(), axis=0)

In [33]:
def eqm_callback(prob, i):
    curved_linkage_view.update(scalarField = colors)

In [34]:
elastic_rods.compute_equilibrium(curved_linkage, callback = eqm_callback, options = OPTS)

0	0.000293511	9.79921e-08	9.79921e-08	0	0


In [39]:
def visualize_crossing(index, linkage, view):
    vf = show_selected_joints(linkage, [index])
    view.update(vectorField = vf, scalarField = colors)

In [40]:
ribbons = order_segments_by_ribbons(curved_linkage)
_, _, _, _, all_joint_index, _ = get_turning_angle_and_length_from_ordered_rods(ribbons, curved_linkage, rest = True)
all_joint_index_list = [j_list + [j_list[0]] for j_list in all_joint_index]

def get_ribbon_crossing_list(index, linkage, view):
    selected_list = []
    selected_ribbon = []
    for ribbon_index, index_list in enumerate(all_joint_index_list):
        if index in set(index_list):
            selected_ribbon.append(ribbon_index)
            selected_list.append(index_list)
    print("The crossing {} belongs to ribbon {}".format(index, ', '.join([str(x) for x in selected_ribbon])))
    for i in range(len(selected_list)):
        print('Ribbon {}: {}'.format(selected_ribbon[i], selected_list[i]))

In [41]:
def select_crossing(crossing_index):
    visualize_crossing(crossing_index, curved_linkage, curved_linkage_view)
    get_ribbon_crossing_list(crossing_index, curved_linkage, curved_linkage_view)
select_crossing(0)

The crossing 0 belongs to ribbon 0, 2
Ribbon 0: [0, 1, 155, 175, 174, 186, 202, 204, 206, 210, 209, 224, 221, 38, 37, 45, 56, 60, 81, 85, 94, 100, 120, 145, 144, 142, 140, 4, 3, 26, 179, 180, 182, 183, 197, 198, 215, 216, 219, 223, 222, 43, 42, 50, 59, 65, 84, 99, 98, 97, 147, 148, 150, 141, 125, 28, 21, 22, 172, 173, 162, 171, 194, 195, 218, 227, 226, 236, 237, 48, 47, 64, 63, 62, 103, 104, 106, 107, 111, 112, 128, 123, 122, 24, 16, 17, 166, 164, 167, 168, 191, 192, 230, 231, 234, 241, 240, 251, 250, 68, 69, 70, 75, 76, 88, 89, 110, 115, 127, 130, 129, 19, 11, 12, 160, 161, 151, 159, 187, 188, 233, 238, 244, 245, 247, 248, 34, 35, 53, 54, 74, 79, 87, 92, 114, 118, 132, 134, 133, 14, 6, 7, 156, 153, 157, 176, 190, 203, 200, 208, 211, 212, 225, 32, 31, 40, 52, 57, 78, 82, 91, 95, 117, 121, 136, 138, 137, 9, 0]
Ribbon 2: [2, 3, 29, 125, 124, 128, 109, 110, 86, 87, 77, 78, 55, 56, 41, 42, 49, 237, 235, 234, 232, 233, 189, 190, 177, 174, 178, 179, 27, 21, 25, 122, 126, 127, 113, 114, 90, 9